In [3]:
import random
import numpy as np
import pandas as pd
import math
import openpyxl
import time
import sys
np.set_printoptions(threshold=sys.maxsize)
import xlwings
filename1 = "C:\\Users\\admin\\PYTHON\\Thesis_Meta\\GA data.xlsx"
wb1=xlwings.Book(filename1)
sheet1=wb1.sheets[0]

In [4]:
#paramters
period=int(sheet1.range("D3").value)
supermarket=int(sheet1.range("D4").value)
item = int(sheet1.range("D5").value)
pc=sheet1.range("D6:F6").value
sc=sheet1.range("D9:F9").value
hc=sheet1.range("D12:F12").value
oc=int(sheet1.range("D15").value)
Q_0=sheet1.range("D16:F16").value
WU=int(sheet1.range("D19").value)
WL=int(sheet1.range("D20").value)
I_0=sheet1.range("D21:L21").value
invest=int(sheet1.range("D22").value)
m = 100  #number of chromosomes - population size
n = (3, 3, 12)
shape = (3,3,12)
BigM = 99999999999
# Parameter of GA
pselect = 0.2
pcrossover=0.5
pmutation = 0.3 
maxI_ga = 100
# Starting time
start_time = time.time()

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
#Defuzzification 
a=sheet1.range("B27:DE27").value
b=sheet1.range("B28:DE28").value
c=sheet1.range("B29:DE29").value
a = np.array(a)
b = np.array(b)
c = np.array(c)
calculated_values=((a + 2 * b + c)/4)
Demand=np.ceil(calculated_values.reshape((3,3,12)))

In [ ]:
def initialize(m, n, p):
    v = np.zeros((m, n, p), dtype=int)
    for i in range(m):
        for j in range(n):
            for k in range(p):
                v[i][j][k] = np.random.randint(0, 2, dtype=int)
    return v
shape = (3, 3, 12)
v_list = [initialize(*shape) for _ in range(m)]
print(v_list)

In [ ]:
def check_chromosome(v_list, Demand, WL, WU):
    modified_v_list = []
    cumulative_Demand_list = []
    for v in v_list:
        v_copy = v.copy()
        cumulative_Demand = np.zeros_like(Demand)
        demand_copy = Demand.copy()  # Create a copy of Demand to preserve the original values
        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                # Initialize the cumulative demand for the first period
                cumulative_Demand[i, j, 0] = demand_copy[i, j, 0]
                for k in range(1, v_copy.shape[2]):
                    if v_copy[i, j, k] == 1:
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]
                    else:
                        cumulative_Demand[i, j, k] = cumulative_Demand[i, j, k-1] + demand_copy[i, j, k]
                    
                    # Check if the cumulative demand exceeds the upper limit
                    if cumulative_Demand[i, j, k] > WU:
                        v_copy[i, j, k] = 1
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]

                    # Adjust v and cumulative demand based on the lower limit
                    if cumulative_Demand[i, j, k] <= WL:
                        v_copy[i, j, k] = 0
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]  # Update with current period's demand
                        if cumulative_Demand[i, j, k] > WU:
                            v_copy[i, j, k] = 1
                            cumulative_Demand[i, j, k] = demand_copy[i, j, k]  # Update with current period's demand
        modified_v_list.append(v_copy)
        cumulative_Demand_list.append(cumulative_Demand)
    return modified_v_list
modified_v_list = check_chromosome(v_list, Demand, WL, WU)


In [ ]:
def CummuD(v_list, Demand, WL, WU):
    modified_v_list = []
    cumulative_Demand_list = []
    for v in v_list:
        v_copy = v.copy()
        cumulative_Demand = np.zeros_like(Demand)
        demand_copy = Demand.copy()  # Create a copy of Demand to preserve the original values
        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                # Initialize the cumulative demand for the first period
                cumulative_Demand[i, j, 0] = demand_copy[i, j, 0]
                for k in range(1, v_copy.shape[2]):
                    if v_copy[i, j, k] == 1:
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]
                    else:
                        cumulative_Demand[i, j, k] = cumulative_Demand[i, j, k-1] + demand_copy[i, j, k]
                    
                    # Check if the cumulative demand exceeds the upper limit
                    if cumulative_Demand[i, j, k] > WU:
                        v_copy[i, j, k] = 1
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]

                    # Adjust v and cumulative demand based on the lower limit
                    if cumulative_Demand[i, j, k] <= WL:
                        v_copy[i, j, k] = 0
                        cumulative_Demand[i, j, k] = demand_copy[i, j, k]  # Update with current period's demand
                        if cumulative_Demand[i, j, k] > WU:
                            v_copy[i, j, k] = 1
                            cumulative_Demand[i, j, k] = demand_copy[i, j, k]  # Update with current period's demand
        
        modified_v_list.append(v_copy)
        cumulative_Demand_list.append(cumulative_Demand)
    return cumulative_Demand_list

# Check the chromosomes and get the modified v list and cumulative demand list
cumulative_Demand_list = CummuD(v_list, Demand, WL, WU)

# Print each modified value of v and the corresponding cumulative demand
#for i, (modified_v, cumulative_Demand) in enumerate(zip(modified_v_list, cumulative_Demand_list), start=1):
#    print(f"Modified v[{i}]:")
 #   print(modified_v)
  #  print(f"Cumulative Demand[{i}]:")
   # print(cumulative_Demand)
    #print("\n")


In [ ]:
def fitness_profi(modified_v_list, Demand, sc, hc, oc, pc):
    total_profit_list = []
    for v in modified_v_list:
        D = np.zeros_like(Demand)
        Q = np.zeros_like(Demand)
        I = np.zeros_like(Demand)
        cumulative_Q = np.zeros_like(Demand)
        Totalprofit = np.zeros(modified_v_list.shape[0])
        Ordering = np.zeros((v.shape[0], v.shape[1]))
        Purchasing = np.zeros((v.shape[0], v.shape[1]))
        Holding = np.zeros((v.shape[0], v.shape[1]))
        Selling = np.zeros((v.shape[0], v.shape[1]))

        initial_inventory = {
            (0, 0): 12, (0, 1): 10, (0, 2): 6,
            (1, 0): 18, (1, 1): 10, (1, 2): 0,
            (2, 0): 2,  (2, 1): 5,  (2, 2): 12
        }
        for (i, j), inv in initial_inventory.items():
            I[i, j, 0] = inv
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                cumulative_Q[i, j, 0] = 0
                for k in range(v.shape[2]):
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v[i, j, k] = 0
                        Q[i, j, k] = 0
                        cumulative_Q[i, j, k] = 0
                        if k < v.shape[2] - 1:
                            I[i, j, k + 1] = I[i, j, k] - Demand[i, j, k]
                    else:
                        if v[i, j, k] == 1:  # If v=1, set cumulative_Q to Q
                            Q[i, j, k] = Demand[i, j, k]
                            cumulative_Q[i, j, k] = Q[i, j, k]
                        else:  # If v=0, set cumulative_Q to cumulative_Q[k-1] + D
                            if k > 0:
                                cumulative_Q[i, j, k] = cumulative_Q[i, j, k-1] + Demand[i, j, k]
                                Q[i, j, k] = 0  # Set Q to 0 when v=0

                        if k == v.shape[2] - 1:
                            if k < v.shape[2] - 1:
                                I[i, j, k + 1] = I[i, j, k] + Q[i, j, k] - Demand[i, j, k]
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                Ordering[i, j] = oc * np.sum(v[i, j, :])
                Purchasing[i, j] = pc[j] * np.sum(Q[i, j, :])
                Holding[i, j] = hc[j] * np.sum(I[i, j, :])
                Selling[i, j] = np.sum(Demand[i, j, :]) * sc[j]

        for i in range(v.shape[0]):
            Totalprofit[i] = np.sum(Selling[i, :] - Ordering[i, :] - Purchasing[i, :] - Holding[i, :])
        total_profit_list.append(np.sum(Totalprofit))
    return total_profit_list
total_profit = fitness_profi(modified_v_list, Demand, sc, hc, oc, pc)
print(total_profit)

In [ ]:
def dvar(modified_v_list, Demand, sc, hc, oc, pc):
    total_profit_list = []
    for v in modified_v_list:
        D = np.zeros_like(Demand)
        Q = np.zeros_like(Demand)
        I = np.zeros_like(Demand)
        cumulative_Q = np.zeros_like(Demand)
        Totalprofit = np.zeros(v.shape[0])
        Ordering = np.zeros((v.shape[0], v.shape[1]))
        Purchasing = np.zeros((v.shape[0], v.shape[1]))
        Holding = np.zeros((v.shape[0], v.shape[1]))
        Selling = np.zeros((v.shape[0], v.shape[1]))
    
        initial_inventory = {
            (0, 0): 12, (0, 1): 10, (0, 2): 6,
            (1, 0): 18, (1, 1): 10, (1, 2): 0,
            (2, 0): 2,  (2, 1): 5,  (2, 2): 12
        }
        for (i, j), inv in initial_inventory.items():
            I[i, j, 0] = inv
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                cumulative_Q[i, j, 0] = 0
                for k in range(v.shape[2]):
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v[i, j, k] = 0
                        Q[i, j, k] = 0
                        cumulative_Q[i, j, k] = 0
                        if k < v.shape[2] - 1:
                            I[i, j, k + 1] = I[i, j, k] - Demand[i, j, k]
                    else:
                        if v[i, j, k] == 1:  # If v=1, set cumulative_Q to Q
                            Q[i, j, k] = Demand[i, j, k]
                            cumulative_Q[i, j, k] = Q[i, j, k]
                        else:  # If v=0, set cumulative_Q to cumulative_Q[k-1] + D
                            if k > 0:
                                cumulative_Q[i, j, k] = cumulative_Q[i, j, k-1] + Demand[i, j, k]
                                Q[i, j, k] = 0  # Set Q to 0 when v=0
    
                        if k == v.shape[2] - 1:
                            if k < v.shape[2] - 1:
                                I[i, j, k + 1] = I[i, j, k] + Q[i, j, k] - Demand[i, j, k]
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                for k in range(v.shape[2]):
                    Ordering[i, j] = oc * np.sum(v[i, j, :])
                    Purchasing[i, j] = pc[j] * np.sum(Q[i, j, :])
                    Holding[i, j] = hc[j] * np.sum(I[i, j, :])
                    Selling[i, j] = np.sum(Demand[i, j, :]) * sc[j]
    
        for i in range(v.shape[0]):
            Totalprofit[i] = np.sum(Selling[i, :] - Ordering[i, :] - Purchasing[i, :] - Holding[i, :])
    return I, Q, cumulative_Q, Totalprofit, Ordering, Purchasing, Holding, Selling

In [ ]:
def fitness_profit(modified_v_list, Demand, sc, hc, oc, pc):
    D = np.zeros_like(Demand)
    Q = np.zeros_like(Demand)
    I = np.zeros_like(Demand)
    cumulative_Q = np.zeros_like(Demand)
    Totalprofit = np.zeros(v.shape[0])
    Ordering = np.zeros((v.shape[0], v.shape[1]))
    Purchasing = np.zeros((v.shape[0], v.shape[1]))
    Holding = np.zeros((v.shape[0], v.shape[1]))
    Selling = np.zeros((v.shape[0], v.shape[1]))

    initial_inventory = {
        (0, 0): 12, (0, 1): 10, (0, 2): 6,
        (1, 0): 18, (1, 1): 10, (1, 2): 0,
        (2, 0): 2,  (2, 1): 5,  (2, 2): 12
    }
    for (i, j), inv in initial_inventory.items():
        I[i, j, 0] = inv
    for i in range(v.shape[0]):
        for j in range(v.shape[1]):
            cumulative_Q[i, j, 0] = 0
            for k in range(v.shape[2]):
                if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                    v[i, j, k] = 0
                    Q[i, j, k] = 0
                    cumulative_Q[i, j, k] = 0
                    if k < v.shape[2] - 1:
                        I[i, j, k + 1] = I[i, j, k] - Demand[i, j, k]
                else:
                    if v[i, j, k] == 1:  # If v=1, set cumulative_Q to Q
                        Q[i, j, k] = Demand[i, j, k]
                        cumulative_Q[i, j, k] = Q[i, j, k]
                    else:  # If v=0, set cumulative_Q to cumulative_Q[k-1] + D
                        if k > 0:
                            cumulative_Q[i, j, k] = cumulative_Q[i, j, k-1] + Demand[i, j, k]
                            Q[i, j, k] = 0  # Set Q to 0 when v=0

                    if k == v.shape[2] - 1:
                        if k < v.shape[2] - 1:
                            I[i, j, k + 1] = I[i, j, k] + Q[i, j, k] - Demand[i, j, k]
    for i in range(v.shape[0]):
        for j in range(v.shape[1]):
            Ordering[i, j] = oc * np.sum(v[i, j, :])
            Purchasing[i, j] = pc[j] * np.sum(Q[i, j, :])
            Holding[i, j] = hc[j] * np.sum(I[i, j, :])
            Selling[i, j] = np.sum(Demand[i, j, :]) * sc[j]

    for i in range(v.shape[0]):
        Totalprofit[i] = np.sum(Selling[i, :] - Ordering[i, :] - Purchasing[i, :] - Holding[i, :])
    return np.sum(Totalprofit)

In [ ]:
def rank_fitness(modified_v_list, Demand, sc, hc, oc, pc):
    fitness_values = []
    for i, modified_v in enumerate(modified_v_list, start=1):
        profit = fitness_profi(modified_v_list, Demand, sc, hc, oc, pc)
        fitness_values.append((i, profit))
    
    # Sort the fitness values from highest to lowest
    ranked_fitness = sorted(fitness_values, key=lambda x: x[1], reverse=True)
    return ranked_fitness

In [ ]:
def rank_pop(modified_v_list, Demand, sc, hc, oc, pc):
    fitness_values = []
    for i, modified_v in enumerate(modified_v_list, start=1):
        profit = fitness_profi(modified_v_list, Demand, sc, hc, oc, pc)
        fitness_values.append((i, profit))
    # Sort the fitness values from highest to lowest
    ranked_fitness = sorted(fitness_values, key=lambda x: x[1], reverse=True)
    
    # Extract only the indices in the order of ranking
    ranked_indices = [index for index, profit in ranked_fitness]
    return ranked_indices

In [ ]:
def ga_generate(ranked_indices, m, shape, pselect, pmutation):
    n, o, p = shape  # Dimensions of each chromosome
    popu = np.zeros((m, n, o, p), dtype=int)  # New population
    ms = round(m * pselect)  # Number of selected parents
    mm = ms + round((m - ms) * pmutation)  # Number of mutated individuals
    parent = np.zeros((ms, n, o, p), dtype=int)  # Array to store selected parents
    
    # Copy values from modified_v_list to parent and popu arrays
    for i in range(ms):
        parent[i] = modified_v_list[ranked_indices[i]]  # Assign ranked individuals to parent
        popu[i] = parent[i]  # Copy to popu as well
    
    # Mutating some individuals
    for i in range(ms, mm):
        j = i - ms
        if j < ms:
            popu[i] = parent[j]
        else:
            j = np.random.randint(0, ms, dtype=int)
            popu[i] = parent[j]
        
        # Randomly select mutation points within the chromosome
        t = np.random.randint(1, p, dtype=int)
        l = random.choice(list(range(1, t)) + list(range(t + 1, p)))
        
        # Mutation
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][t] == 0:
            popu[i][x][y][t] = 1
        else:
            popu[i][x][y][t] = 0
        
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][l] == 0:
            popu[i][x][y][l] = 1
        else:
            popu[i][x][y][l] = 0
    # Crossover
    for i in range(mm, m-1, 2):
        par1 = np.random.randint(0, ms, dtype=int)
        popu[i] = parent[par1]
        
        par2 = random.choice(list(range(0, par1)) + list(range(par1 + 1, ms)))
        popu[i + 1] = parent[par2]
        
        # Randomly select crossover point within the chromosome
        p_cross = np.random.randint(1, p, dtype=int)
        x_cross, y_cross = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        for a in range(p_cross):
            popu[i][x_cross][y_cross][a] = parent[par2][x_cross][y_cross][a]
            popu[i + 1][x_cross][y_cross][a] = parent[par1][x_cross][y_cross][a]
    
    return popu

In [ ]:
print("GENETIC ALGORITHM OPMIZATION")
iter = 0
v_list = [initialize(*shape) for _ in range(m)]
modified_v_list = check_chromosome(v_list, Demand, WL, WU)
profit = fitness_profi(modified_v_list, Demand, sc, hc, oc, pc)
ranked_fitness = rank_fitness(modified_v_list, Demand, sc, hc, oc, pc)
ranked_indices = rank_pop(modified_v_list, Demand, sc, hc, oc, pc)
new_population = ga_generate(ranked_indices, m, shape, pselect, pmutation)
while iter < maxI_ga:
    iter = iter +1
    modified_v_list = check_chromosome(v_list, Demand, WL, WU)
    profit = fitness_profi(modified_v_list, Demand, sc, hc, oc, pc)
    ranked_fitness = rank_fitness(modified_v_list, Demand, sc, hc, oc, pc)
    ranked_indices = rank_pop(modified_v_list, Demand, sc, hc, oc, pc)
    new_population = ga_generate(ranked_indices, m, shape, pselect, pmutation)
# Print the best solution and time taken
best_solution_index = ranked_indices[0]
best_solution_fitness = ranked_fitness[0][1]
print(f"Iteration {iter}: Best solution - v[{best_solution_index}] with profit {best_solution_fitness}")
# Ending time
end_time = time.time()
print("Time taken to run the code:", end_time - start_time, "seconds")
print("Time taken to run the code:", (end_time - start_time) / 3600, "hours")